In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import math
import time

df = pd.DataFrame(columns = ['Start', 'Number of Links','Searches','Jump/Loop Link','Efficiency','Time','Time/Search'])
df.head()


def findLink(paragraph):
    #print(paragraph)
    if 'href="/wiki' in paragraph:
        #print(paragraph)
        #print("theres a link")
        while paragraph.find("(") < paragraph.find('href="/wiki/'):
            #print(paragraph.find("("))
            #print(paragraph.find('href="/wiki/'))
            if paragraph.find("(") == -1:
                break
            opening = 1
            closing = 0
            paragraph = paragraph[paragraph.find("(")+1:]
            while opening != closing:
                openingIndex = paragraph.find("(")
                if openingIndex == -1:
                    openingIndex = 100000000
                if openingIndex < paragraph.find(")"):
                    opening += 1
                    paragraph = paragraph[paragraph.find("(")+1:]
                else:
                    closing += 1
                    paragraph = paragraph[paragraph.find(")")+1:]
        #print(paragraph)
        try:
          #only using try if the only href in the paragraph is within parentheses
          return(paragraph.split('href="/wiki/')[1].split('"')[0])
        except:
          return("")
    else:
        return("")

def strip_tables(soup):
  count = 0
  #print(count)
  while count < 5:
    #print("searching for table")
    if soup.find('<table') == -1:
      #print("no more tables")
      return soup
      break
    #print(soup.find('<table'))
    #print("Found table")
    soup = soup[:soup.find('<table')] + soup[soup.find('table>') + 6 : ]
    count += 1
  return soup

def strip_i(soup):
  count = 0
  while count < 5:
    if soup.find('<i>') == -1:
      return soup
    soup = soup[:soup.find('<i>')] + soup[soup.find('</i>') + 4 : ]
    count += 1
  return soup

def strip_coordinates(soup):
    if soup.find('span id="coordinates"><a href="/wiki/Geographic_coordinate_system"') == -1:
      return soup
    else:
        soup = soup[:soup.find('span id="coordinates"><a href="/wiki/Geographic_coordinate_system"')] + soup[soup.find('span id="coordinates"><a href="/wiki/Geographic_coordinate_system"') + 250 : ]
        return soup

commonLinks = {"Science" : 6,
              "Country" : 11,
              "Sport" : 16,
              "Animal" : 12,
              "Sovereign_state": 12,
              "Continent" : 16,
              "Language" : 6,
              "Machine" : 16}
      
for x in range(10):
  startTime = time.time()
  print("-----------------------------------")
  print("---------------  " + str(x+1) + "  ---------------")
  print("-----------------------------------")
  link = 'Special:Random'
  page = requests.get('http://en.wikipedia.org/wiki/' + link)
  #print("Page Loaded")
  soup = BeautifulSoup(page.content, 'html.parser')
  numberOfLinks = 0
  searches = 0
  titles = []
  jumpLink = "N/A"
  loopTrigger = "N/A"
  while link != 'Philosophy':
      if link in commonLinks:
        numberOfLinks += commonLinks[str(link)]
        jumpLink = str(link)
        break
      #print("link:", link)
      page = requests.get('http://en.wikipedia.org/wiki/' + link)
      #print("Page Loaded")
      soup = BeautifulSoup(page.content, 'html.parser')
      title = soup.find('h1',attrs={'class':'firstHeading'}).get_text()
      if title in titles:
        print("Loop Encountered:", str(link))
        numberOfLinks = "N/A : Loop"
        loopTrigger = str(link)
        break
      titles.append(title)
      print(title, "["+str(numberOfLinks)+"]")
      ### strips the first 4 tables from the soup 
      soup = str(soup)
      soup = strip_tables(soup)  
      soup = strip_i(soup)
      soup = strip_coordinates(soup)
      #print ('stripped tables')

      regex = r"<p.+"
      list_of_paragraphs = re.findall(regex,str(soup))
      #print(list_of_paragraphs)
      
      
      ### Extend the text of the unordered lists if the link turns out to be blank      
      regex = r"<li>.+<\/li>"
      list_of_paragraphs.extend(re.findall(regex,str(soup)))

      link = ""
      #print(list_of_paragraphs)
      for paragraph in list_of_paragraphs:
          link = findLink(paragraph)
          #print(link)
          if link != "":
              break

      #link = findLink(paragraph)
      numberOfLinks += 1
      searches += 1
  print("-----------------------------------")
  endTime = time.time()
  timeTaken = round((endTime - startTime),2)
  timePerSearch = round((timeTaken/searches),2)
  percentSaved = "0%"
  if numberOfLinks == "N/A : Loop":
    print('Philosophy cannot be reached from', titles[0])
    df.loc[x] = [titles[0], numberOfLinks, searches, loopTrigger, percentSaved, timeTaken, timePerSearch]
  else:
    print('Philosophy is', numberOfLinks, 'links From', titles[0])
    if jumpLink != "N/A":
      print("Jumped from:", jumpLink)
      percentSaved = (str(100 - math.ceil((searches/numberOfLinks)*100))+ " %")
      print("Saved", numberOfLinks - searches, "searches, or", percentSaved)

    # add the number of clicks to the df
    df.loc[x] = [titles[0], numberOfLinks, searches, jumpLink, percentSaved, timeTaken, timePerSearch]
display(df)